In [ ]:
!pip install -U sentence_transformers --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
#First, we load the  dataset (with query and answer information)
# Indicate the dataset id from the Hub
dataset_id = "sentence-transformers/natural-questions"
dataset_file = load_dataset(dataset_id, split="train")

In [ ]:
print(dataset_file[0])

{'query': 'when did richmond last play in a preliminary final', 'answer': "Richmond Football Club Richmond began 2017 with 5 straight wins, a feat it had not achieved since 1995. A series of close losses hampered the Tigers throughout the middle of the season, including a 5-point loss to the Western Bulldogs, 2-point loss to Fremantle, and a 3-point loss to the Giants. Richmond ended the season strongly with convincing victories over Fremantle and St Kilda in the final two rounds, elevating the club to 3rd on the ladder. Richmond's first final of the season against the Cats at the MCG attracted a record qualifying final crowd of 95,028; the Tigers won by 51 points. Having advanced to the first preliminary finals for the first time since 2001, Richmond defeated Greater Western Sydney by 36 points in front of a crowd of 94,258 to progress to the Grand Final against Adelaide, their first Grand Final appearance since 1982. The attendance was 100,021, the largest crowd to a grand final sinc

In [ ]:
import os
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset


# Use the allenai-specter model with SentenceTransformers
model = SentenceTransformer('allenai-specter')

# Prepare paper texts by combining query and answer fields
paper_texts = [
    record['query'] + '[SEP]' + record['answer'] for record in dataset_file.select(range(32))
]

# Compute embeddings for all paper texts
corpus_embeddings = model.encode(paper_texts, convert_to_tensor=True, show_progress_bar=True)

# Function to search for answers given a query
def search_papers(query):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Perform semantic search
    search_hits = util.semantic_search(query_embedding, corpus_embeddings)
    search_hits = search_hits[0]  # Get the hits for the first query

    print("\n\nQuery:", query)
    print("Most similar answers:")
    for hit in search_hits[:5]:  # Limit to top 5 results for clarity
        related_text = dataset_file[int(hit['corpus_id'])]  # Access related record
        print("{:.2f}\tAnswer: {}".format(
            hit['score'], related_text['answer']
        ))

# Example usage
search_papers("when did richmond last play in a preliminary final")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]



Query: when did richmond last play in a preliminary final
Most similar answers:
0.87	Answer: Jack Scott (singer) At the beginning of 1960, Scott again changed record labels, this time to Top Rank Records.[1] He then recorded four Billboard Hot 100 hits – "What in the World's Come Over You" (#5), "Burning Bridges" (#3) b/w "Oh Little One" (#34), and "It Only Happened Yesterday" (#38).[1] "What in the World's Come Over You" was Scott's second gold disc winner.[6] Scott continued to record and perform during the 1960s and 1970s.[1] His song "You're Just Gettin' Better" reached the country charts in 1974.[1] In May 1977, Scott recorded a Peel session for BBC Radio 1 disc jockey, John Peel.
0.86	Answer: Cooley High Cooley High is a 1975 American coming-of-age/ drama film that follows the narrative of high school seniors and best-friends, Leroy "Preach" Jackson (Glynn Turman) and Richard "Cochise" Morris (Lawrence Hilton-Jacobs). Written by Eric Monte, directed by Michael Schultz and produ

In [39]:
search_hits = search_papers("who made the song my achy breaky heart")





Query: who made the song my achy breaky heart
Most similar answers:
0.89	Answer: Achy Breaky Heart "Achy Breaky Heart" is a country song written by Don Von Tress. Originally titled "Don't Tell My Heart" and performed by The Marcy Brothers in 1991, its name was later changed to "Achy Breaky Heart" and performed by Billy Ray Cyrus on his 1992 album Some Gave All. The song is Cyrus' debut single and signature song, it made him famous and has been his most successful song. It became the first single ever to achieve triple Platinum status in Australia[1] and also 1992's best-selling single in the same country.[2][3] In the United States it became a crossover hit on pop and country radio, peaking at number 4 on the Billboard Hot 100 and topping the Hot Country Songs chart, becoming the first country single to be certified Platinum since Kenny Rogers and Dolly Parton's "Islands in the Stream" in 1983.[4] The single topped in several countries, and after being featured on Top of the Pops in 

In [ ]:
from transformers import pipeline

# Summarization pipeline
summarizer = pipeline("summarization")

# Collect the relevant answers from the search function
def search_papers_and_summarize(query, max_summary_length=45):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Perform semantic search
    search_hits = util.semantic_search(query_embedding, corpus_embeddings)
    search_hits = search_hits[0]  # Get the hits for the first query

    # Collect answers from top hits
    answers = []
    for hit in search_hits[:5]:  # Limit to top 5 results
        related_text = dataset_file[int(hit['corpus_id'])]
        answers.append(related_text['answer'])

    # Combine answers into a single text for summarization
    combined_text = " ".join(answers)

    # Summarize the combined text
    summary = summarizer(combined_text, max_length=max_summary_length, clean_up_tokenization_spaces=True)
    print("Summary:")
    print(summary[0]['summary_text'])

# Example usage
search_papers_and_summarize("who made the song my achy breaky heart")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your min_length=56 must be inferior than your max_length=45.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (45). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


Summary:
 Billy Ray Cyrus' "Achy Breaky Heart" is his signature song. It is considered by some as one of the worst songs of all time, featuring at number two in VH1 and Blender


In [ ]:
search_papers_and_summarize("Who is wimpy kid")

Your min_length=56 must be inferior than your max_length=45.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (45). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


Summary:
 The show first premiered on Cartoon Network on August 13, 2004, as a 90-minute television film. The series finished its run on May 3, 2009, with a total of six seasons and seventy-nine
